In [ ]:
import pandas as pd
import yaml
import sqlite3

customer_statistics = '/content/sample_data/customer_statistics.csv'
orders = '/content/sample_data/orders.csv'
customer_demographics = '/content/sample_data/customer_demographics.yaml'


with open(customer_demographics, 'r') as file:
    data = yaml.safe_load(file)

df_customer_demographics = pd.DataFrame(data).transpose()
df_customer_statistics = pd.read_csv(customer_statistics)
df_orders = pd.read_csv(orders)


In [ ]:
#clean
df_customer_statistics = df_customer_statistics.groupby('customer_id').agg({
    'total_orders': 'sum',
    'total_items': 'sum',
    'total_spent': 'sum'}).reset_index()

In [ ]:
#merge first round
merged_final = pd.merge(df_customer_statistics, df_customer_demographics, left_on='customer_id', right_index=True, how='inner')

In [ ]:
#merge second round
merged_final = pd.merge(merged_final, df_orders, on='customer_id', how='right')

In [ ]:
conn = sqlite3.connect("orders.db")
cursor = conn.cursor()

stmnt1 = '''create table orders (
    order_id text primary key,
    customer_id integer,
    items text,
    aperitifs text,
    appetizers text,
    entrees text,
    desserts text,
    total numeric(10, 2)
    )
'''
stmnt2 = '''create table customers (
    id integer primary key,
    total_orders int,
    total_items int,
    total_spent numeric(10, 2),
    address text,
    city text,
    credit_card_expires date,
    credit_card_number text,
    credit_card_security_code text,
    credit_card_provider text,
    customer_id text,
    email text,
    name text,
    phone_number text,
    state text,
    zip_code text check (length(zip_code) = 5)
    )
'''
cursor.execute(stmnt1)
cursor.execute(stmnt2)

In [ ]:
merged_list = merged_final.values.tolist()
customer_list = []
orders_list = []
id_set = set()

for row in merged_list:
    if not pd.isna(row[1]) and row[0] not in id_set:
        customer_list.append(row[1:17])
        id_set.add(row[0])
    orders_list.append(row[17::] + [row[0]])

In [ ]:
stmnt3 = 'insert into orders (order_id, items, aperitifs, appetizers, entrees, desserts, total, customer_id) values (?, ?, ?, ?, ?, ?, ?, ?)'
cursor.executemany(stmnt3, orders_list)

In [ ]:
stmnt4 = 'insert into customers (id, total_orders, total_items, total_spent, address, city, credit_card_expires, credit_card_number, credit_card_provider, credit_card_security_code, customer_id, email, name, phone_number, state, zip_code) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'
cursor.executemany(stmnt4, customer_list)

In [ ]:
conn.commit()